In [1]:
from unsloth import FastLanguageModel

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/data/common/ethanchang/broad_and_narrow/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.9: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 8. Max memory: 47.408 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [11]:
task_suite = [
    {
        "name": "English to French Translation",
        "instruction": "Translate the following from English to French.",
        "examples": [
            "the cat is black -> le chat est noir",
            "the dog is white -> le chien est blanc",
            "I love science -> j'aime la science",
            "are you my friend? ->"
        ]
    },
    {
        "name": "Country to Capital",
        "instruction": "What is the capital of the following country?",
        "examples": [
            "Japan -> Tokyo",
            "Germany -> Berlin",
            "Canada -> Ottawa",
            "France -> "
        ]
    },
    {
        "name": "Simple Addition",
        "instruction": "Calculate the sum of these two numbers.",
        "examples": [
            "5 + 8 -> 13",
            "10 + 4 -> 14",
            "7 + 2 -> 9",
            "3 + 6 -> "
        ]
    },
    {
        "name": "Unscramble Words",
        "instruction": "Unscramble the following letters to form a word.",
        "examples": [
            "pleap -> apple",
            "nbaaan -> banana",
            "rycher -> cherry",
            "clinep -> "
        ]
    },
    {
		"name": "minus -> plus",
		"instruction": "Identify the pattern in the following math operations.",
		"examples": [
			"3 - 2 -> 5",
			"8 - 5 -> 13",
			"3 - 6 -> 9",
			"10 - 4 -> "
		]
	}
]

In [12]:
for task in task_suite:
    task_name = task["name"]
    print(f"\nProcessing task: {task_name}")

    # # Clear previous run's data
    # all_layer_outputs.clear()

    # Prepare inputs for the current task
    text_to_process = [task["instruction"]] + task["examples"]
    print(text_to_process)


Processing task: English to French Translation
['Translate the following from English to French.', 'the cat is black -> le chat est noir', 'the dog is white -> le chien est blanc', "I love science -> j'aime la science", 'are you my friend? ->']

Processing task: Country to Capital
['What is the capital of the following country?', 'Japan -> Tokyo', 'Germany -> Berlin', 'Canada -> Ottawa', 'France -> ']

Processing task: Simple Addition
['Calculate the sum of these two numbers.', '5 + 8 -> 13', '10 + 4 -> 14', '7 + 2 -> 9', '3 + 6 -> ']

Processing task: Unscramble Words
['Unscramble the following letters to form a word.', 'pleap -> apple', 'nbaaan -> banana', 'rycher -> cherry', 'clinep -> ']

Processing task: minus -> plus
['Identify the pattern in the following math operations.', '3 - 2 -> 5', '8 - 5 -> 13', '3 - 6 -> 9', '10 - 4 -> ']


In [13]:
test = ['Identify the pattern in the following math operations.', '3 - 2 -> 5', '8 - 5 -> 13', '3 - 6 -> 9', '10 - 4 -> ']
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token
inputs = tokenizer(test, padding=True, return_tensors="pt").to("cuda")
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


ValueError: TextStreamer only supports batch size 1

In [ ]:
# from transformers.generation.streamers import TextStreamer
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [9]:
messages = [
                               # EDIT HERE!
    {"from": "human", "value" : 
"""Identify the pattern in the following math operations.
3 - 2 -> 5
8 - 5 -> 13
3 - 6 -> 9
10 - 4 ->"""
 	}

]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

Identify the pattern in the following math operations.
3 - 2 -> 5
8 - 5 -> 13
3 - 6 -> 9
10 - 4 -><|eot_id|><|start_header_id|>assistant<|end_header_id|>

To identify the pattern, let's analyze the operations:

1. 3 - 2 = 1 (result is 1 more than 3)
2. 8 - 5 = 3 (result is 3 more than 8)
3. 3 - 6 = 3 (result is 3 more than 3)

It seems that the pattern is adding the absolute difference of the two numbers to the larger number. 

So, if we apply this pattern to the next operation:

10 - 4
The absolute difference is 6 (|10 - 4| = 6)
Add 6 to the larger number (10): 10 + 6 = 16

Therefore, the result is 16.<|eot_id|>


In [3]:
messages = [
                               # EDIT HERE!
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>



LlamaForCausalLM has no `_prepare_4d_causal_attention_mask_with_cache_position` method defined in its base modeling class. Compiled forward passes will be sub-optimal. If you're writing code, see Llama for an example implementation. If you're a user, please report this issue on GitHub.


The Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding numbers. The next numbers in the sequence would be:

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144,...<|eot_id|>


In [ ]:
messages = [
    {"from": "human", "value": "Describe the tallest tower in the world."},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

In [ ]:
messages = [
    {"from": "human", "value": "What is Unsloth?"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

In [ ]:
messages = [
    {"from": "human", "value": 'You are a mathematician. Assuming that all numbers are in base-10 where the digits are "0123456789", what is 760+587? Let\'s think step by step, and end the response with the result in "\\boxed{result}".'},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)